In [1]:
import cv2
import numpy as np
import os

In [2]:

video_folder = "/home/persie/Videos/mokap/"
session_id = os.listdir(video_folder)
video_file = []
for folder in session_id:
    files = os.listdir(os.path.join(video_folder, folder))
    for file in files:
        video_file.append(os.path.join(video_folder, folder, file))


In [24]:
def segment_seeds(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return

    # Convert to grayscale
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    
    # Create a mask of the same size as the frame
    mask = np.zeros_like(prev_gray)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate dense optical flow using Farneback method
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        # Calculate magnitude and angle of 2D vectors
        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Threshold the magnitude to identify significant movement
        movement_mask = (magnitude > 1).astype(np.uint8) * 255
        
        # Update the cumulative mask
        mask = cv2.bitwise_or(mask, movement_mask)
        
        # Apply morphological operations to clean up the mask
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filter contours based on area to ignore small movements
        min_area = 100  # Adjust this value based on the size of your seeds
        seed_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

        # Draw the seed contours on the original frame
        result = frame.copy()
        cv2.drawContours(result, seed_contours, -1, (0, 255, 0), 2)

        # Display the result
        cv2.imshow('Segmented Seeds', result)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

        # Update previous frame
        prev_gray = gray

    cap.release()
    cv2.destroyAllWindows()



In [5]:
#Initialise Background
# To initialise the background, I'm going to take the median value of the video pixels
def initialise_background(video_path):
    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        #return
        
    # Convert to grayscale
    ret, first_frame = cap.read()
    frame_array = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    
    sample_rate = 15
    fno = 1
    while ret:
        if fno % sample_rate == 0:
            ret, frame = cap.retrieve()
            if not ret:
                break
                    # Convert current frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_array = np.dstack((frame_array,gray))
            fno += 1
        else:
            # read next frame
            ret = cap.grab()
            fno += 1
  
    print(frame_array.shape)  
    median = np.median(frame_array, axis=2).astype(dtype=np.uint8)
    mean = np.mean(frame_array, axis=2).astype(dtype=np.uint8)    
    sigma = np.std(frame_array, axis=2).astype(dtype=np.uint8)

    return mean, sigma, median
  

In [19]:
#Background Update
def segment_seeds5(video_path, bgs, variance):

    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        fg_mask = bgs.apply(gray, None, 0.0)
        #masked_fg = cv2.inRange(gray, variance)
        cv2.imshow('Frame', frame)
        cv2.imshow('FG Mask', fg_mask)
        #cv2.imshow('masked fg', )

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


In [7]:
def create_bgs(background_image):
    back_sub = cv2.createBackgroundSubtractorMOG2(history=5, varThreshold=30, detectShadows=True)
        
    #blur_background = cv2.blur(background_image, (20, 20))
    back_sub.apply(background_image, None, 1)
    return back_sub
    

In [31]:
def create_bgs2(video_path, median_frame, clearest_frame):

    cap = cv2.VideoCapture(video_path)    
    # Read the first frame, but discard as first frame is blank
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        #return
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)    
    sample_rate = 15
    
    back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=30, detectShadows=True)
    back_sub.apply(clearest_frame, None, 1)
    #back_sub.apply(median_frame, None, 0.5)
    
    # # Convert to grayscale
    # ret, first_frame = cap.read()    
    # 
    # fno = 1
    # while ret:
    #     if fno % sample_rate == 0:
    #         ret, frame = cap.retrieve()
    #         if not ret:
    #             break
    #                 # Convert current frame to grayscale
    #         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #         back_sub.apply(gray, None, (sample_rate/n_frames))
    #         fno += 1
    #     else:
    #         # read next frame
    #         ret = cap.grab()
    #         fno += 1
  
    return back_sub
    

In [6]:
mean_frame, std_dev_frame, median_frame = initialise_background(video_file[33])


(1080, 1440, 496)


In [39]:
6938.0/15

462.53333333333336

In [32]:
bgs = create_bgs2(video_file[33], median_frame, clear_frame)


In [7]:
cap = cv2.VideoCapture(video_file[33])
cap.set(cv2.CAP_PROP_POS_FRAMES, 134)
ret, clear_frame = cap.read()


In [12]:
segment_seeds5(video_file[33], bgs, std_dev_frame)

NameError: name 'segment_seeds5' is not defined

In [28]:
while True:
    cv2.imshow('mean', mean_frame)
    cv2.imshow('median', median_frame)
    cv2.imshow('std_dev', std_dev_frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [9]:
def outlineseed2(video_path, bgs, mean, var):
    cap = cv2.VideoCapture(video_path)
    
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    kernel = np.ones((3,3),np.uint8)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        fg_mask = bgs.apply(gray, None, 0.0)
        cv2.imshow('fg_mask', fg_mask)
        
        
        # noise removal
        sure_fg = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)[1]
        sure_fg = cv2.erode(sure_fg, kernel, iterations=1)
        

        sure_bg = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY)[1]
        sure_bg = cv2.erode(sure_bg, kernel, iterations=1)
        sure_bg = cv2.dilate(sure_bg, kernel, iterations=1)
        

        cv2.imshow('sure_fg', sure_fg)
        cv2.imshow('sure_bg', sure_bg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        cv2.imshow('unknown', unknown)
        # Marker labelling
        ret, markers = cv2.connectedComponents(sure_fg)
         
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1
        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        markers = cv2.watershed(frame,markers)
        frame[markers == -1] = [255,0,0]
        
        cv2.imshow('frame', frame)
        # 
        # # Calculate dense optical flow using Farneback method
        # flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale=0.7, levels=2, winsize=40, iterations=3, poly_n=7, poly_sigma=1.9, flags=0)
        # 
        # # Calculate magnitude and angle of 2D vectors
        # magnitude, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # 
        # hsv[..., 0] = ang*180/np.pi/2
        # hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        # bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        # 
        # cv2.imshow('frame2', bgr)
        # 
        # # Threshold the magnitude to identify significant movement
        # movement_mask = (magnitude > 1).astype(np.uint8) * 255
        # 
        # # Update the cumulative mask
        # #mask = cv2.bitwise_or(mask, movement_mask)
        # movement = hsv[..., 2]
        # # Apply morphological operations to clean up the mask
        # kernel = np.ones((5,5), np.uint8)
        # movement_close = cv2.morphologyEx(movement, cv2.MORPH_CLOSE, kernel)
        # movement_open = cv2.morphologyEx(movement_close, cv2.MORPH_OPEN, kernel)
        # 
        # # Find contours in the mask
        # contours, _ = cv2.findContours(movement_open, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # 
        # # Filter contours based on area to ignore small movements
        # min_area = 100  # Adjust this value based on the size of your seeds
        # seed_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
        # 
        # # Draw the seed contours on the original frame
        # result = frame.copy()
        # cv2.drawContours(result, seed_contours, -1, (0, 255, 0), 2)
        # 
        # #Display the result
        # cv2.imshow('Segmented Seeds', result)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

        # Update previous frame
        prev_gray = gray

    cap.release()
    cv2.destroyAllWindows()


In [33]:
outlineseed2(video_file[33], bgs, mean_frame, std_dev_frame)

In [3]:
def show_frame(video_path, frame_id):
    cap = cv2.VideoCapture(video_path)
    # Read the first frame
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read video")
        return
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = cap.read()
    if not ret:
        return
    
    while True:
        # ret, frame = cap.read()
        # if not ret:
        #     return
        cv2.imshow('Frame', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()


In [4]:
show_frame(video_file[33], 134)